In [1]:
import numpy as np

In [2]:
a=np.array([1,2,3,4])
a.shape

(4,)

In [3]:
class perceptron_implementation():
    def __init__(self):
        self.neuron_weights = None
        self.bias = None
        self.initialize_weights()
        
    def initialize_weights(self):
        # TODO: 
        # Initialize weights 
        # For perceptrons, it's possible to initialize the weights with 0
        self.neuron_weights=np.zeros((2,))
        self.bias=0
        # END TODO

    def forward_pass(self, x):
        # TODO
        # Implement forward propagation
        output = np.matmul(x,self.neuron_weights)+self.bias
        output=np.int64(output>0)
        # END TODO
        return output

    def perceptron_update_rule(self, input_data, target, prediction, learning_rate = 1):
        # TODO
        # Perform perceptron update rule that is defined above
        # use self.neuron_weights
        d_w=learning_rate*(target-prediction)*input_data
        new_weights = self.neuron_weights+d_w

        # END TODO
        self.neuron_weights = new_weights

    def train(self, input_data, targets):
        """
        input_data: Multi-dimensional array that contains all inputs
        """
        # TODO
        # Call the necessary functions to train a single neuron for the given task
        # Complete the rest of the code to correctly train the model
        print("initial weights:",self.neuron_weights)
        for i in range(input_data.shape[0]):
            prob=self.forward_pass(input_data[i])
            print(i,prob)
            self.perceptron_update_rule(input_data[i],targets[i],prob)
            print("epoch:",i,self.neuron_weights)
        # END TODO

    def inference(self, input_data):
        # TODO
        # Test the trained neuron
        output=self.forward_pass(input_data)
        # END TODO
        return output

In [4]:
perceptron = perceptron_implementation()

# TODO

input_data =np.array([[0,0],[0,1],[1,0],[1,1]])
targets = np.array([0,1,1,0])

# train the corresponding single neuron 
perceptron.train(input_data, targets)
# END TODO
perceptron.inference([0,0])

initial weights: [0. 0.]
0 0
epoch: 0 [0. 0.]
1 0
epoch: 1 [0. 1.]
2 0
epoch: 2 [1. 1.]
3 1
epoch: 3 [0. 0.]


0

In [5]:
prediction=perceptron.inference(input_data)
print(prediction)

[0 0 0 0]


In [6]:
def accuracy(predictions: np.ndarray, targets: np.ndarray, threshold=0.5) -> float:
    # TODO
    # Implement the accuracy metric
    predictions_t=np.int64(predictions>threshold)
    accuracy=(predictions_t==targets)
    print(accuracy)
    accuracy_value=np.sum(accuracy)/accuracy.shape[0]
    # END TODO
    return accuracy_value

# TODO
# Call accuracy function and provide necessary inputs to calculate accuracy
accuracy_value = accuracy(prediction,targets)
print(accuracy_value)
# END TODO

[ True False False  True]
0.5


In [6]:
predictions = perceptron.inference(input_data)
print(predictions)

[0 0 0 0]


In [7]:
class sigmoid_activation_function():

    def forward(self, input_data):
        output = None
        # TODO
        # implement Sigmoid function for the input_data
        output=1/(1+np.exp(-input_data))
        self.output=output
        # END TODO
        return output
    
    def backward(self, gradients):
        # calculate the gradients with help of the derivative
        gradients=gradients*self.output*(1-self.output)
        return gradients

In [8]:
class binary_cross_entropy():

    def forward(self, output, target):
        loss = None
        
        # TODO
        # implement Binary Cross-Entrops loss function for output, target
        self.output=output
        self.target=target
        loss_sum=0
        for i in range(output.shape[0]): 
            loss_single=np.array(-(target[i]*np.log(output[i]))-((1-target[i])*np.log(1-output[i])))
            loss_sum+=loss_single
        loss=loss_sum/output.shape[0]
        # END TODO
        return loss
    
    def backward(self):
        # calculate the gradients with help of the derivative
        gradients=(-self.target/self.output)+((1-self.target)/(1-self.output))
        return gradients

In [9]:
def xavier_initialization(n_incoming: np.ndarray, n_outgoing: np.ndarray) -> np.ndarray:
    """ Returns a numpy array of initialized weights """
    a=np.sqrt(6.)/np.sqrt(n_incoming+n_outgoing)
    weights=np.random.uniform(low=-a, high=a, size=[n_outgoing, n_incoming+1])
#     biases=np.random.uniform(low=-a, high=a, size=[n_outgoing])
    return weights

In [9]:
def test_initialization(n_incoming: np.ndarray, n_outgoing: np.ndarray) -> np.ndarray:
    """ Returns a numpy array of initialized weights """
    weights=np.ones((n_outgoing, n_incoming+1))
#     biases=np.random.uniform(low=-a, high=a, size=[n_outgoing])
    return weights

In [17]:
class multi_neuron_implementation():
    def __init__(self, number_of_neurons, loss_function, output_activation_function):
        self.neuron_weights = None
        self.number_of_neurons = number_of_neurons
        self.loss_function = loss_function
        self.output_activation_function = output_activation_function

    def forward_pass(self, x):
        # TODO
        # Implement forward propagation
        output = None
        #initialization
        if self.neuron_weights== None:
            self.neuron_weights=[]
            self.neuron_weights.append(xavier_initialization(2,1))
            for i in range(self.number_of_neurons-1):
                self.neuron_weights.append(xavier_initialization(1,1))
            print("initial weights",self.neuron_weights)
        
        input_next=x
        #record output of each layer
        self.outputs=[]
        for i in range(self.number_of_neurons):
            #c:add x_0=1 to fit the bias item
            c=np.ones(input_next.shape[0])
            input_next=np.c_[input_next,c]
            self.outputs.append(input_next)
            #full connection
            input_next=np.matmul(input_next,self.neuron_weights[i].T)
        self.outputs.append(input_next)
        #output layer
        output=self.output_activation_function.forward(input_next)
        #record output of final layer
        self.outputs.append(output)
        # END TODO
        return output

    def backward_pass(self, x):
        # TODO
        # Perform backpropagation by calculating derivative
        output = None
        #x:derivative of loss
        d_h3=self.output_activation_function.backward(x)
        gradient_before=d_h3
        #d_Ws:record derivation of each weight
        d_Ws=[]
        for i in range(self.number_of_neurons): 
            d_Ws.append(self.outputs[-(i+3)]*gradient_before)
            #d_Z
            gradient_before=gradient_before*self.neuron_weights[-(i+1)][0][0:-1]
        output=d_Ws
        # END TODO
        return output

    def update_parameter(self, loss, derivative, learning_rate = 1):
        # TODO
        # Perform weight update
        # use self.neuron_weights
        
        #revise the derivation let it from W0...
        print("derivative",derivative[::-1])
        loss_gradient=loss.backward()
        new_weights = self.neuron_weights+learning_rate*derivative[::-1]
        # END TODO
        self.neuron_weights = new_weights

    def train(self, input_data, targets):
        # TODO
        # Call the necessary functions to train the model with multiple neurons for the given task
        # Complete the rest of the code to correctly train the model
        
        #epoch 100
        for i in range(100):
            for i in range(input_data.shape[0]):
                output_i=self.forward_pass(input_data[i])
                #loss
                loss=self.loss_function
                loss_value=loss.forward(output_i,targets[i])
                loss_gradient=loss.backward()
                derivative=self.backward_pass(loss_gradient)
                self.update_parameter(loss,derivative)
        # END TODO

    def inference(self, input_data):
        # TODO
        # Test the trained model
        output=self.forward_pass(input_data)
        # END TODO
        return output

In [19]:
multi_neuron = multi_neuron_implementation(number_of_neurons=3,loss_function=binary_cross_entropy(),output_activation_function=sigmoid_activation_function())

# TODO
input_data =np.array([[[0,1]],[[0,0]],[[1,0]],[[1,1]]])
targets = np.array([[1],[0],[1],[0]])
# train the corresponding single neuron 
multi_neuron.train(input_data, targets)
# END TODO

initial weights [array([[ 0.3795061 ,  0.42507769, -0.63346948]]), array([[0.82976742, 1.36717329]]), array([[0.29768516, 0.59971431]])]
derivative [array([[-0.        , -0.06862784, -0.06862784]]), array([[ 0.01723553, -0.08270732]]), array([[-0.33180615, -0.27783489]])]
derivative [array([[-0.        , -0.        , -0.00400672]]), array([[ 0.14726163, -0.23246839]]), array([[0.58959629, 0.70061509]])]
derivative [array([[-0.02431958, -0.        , -0.02431958]]), array([[ 0.04194088, -0.16514538]]), array([[-0.32391856, -0.28009909]])]
derivative [array([[-0.01006105, -0.01006105, -0.01006105]]), array([[-0.04104802, -0.23988655]]), array([[1.1176476 , 0.74057675]])]
derivative [array([[0.        , 0.01274629, 0.01274629]]), array([[ 0.06471013, -0.3105215 ]]), array([[-0.33180615, -0.27783489]])]
derivative [array([[-0.        , -0.        , -0.01504306]]), array([[ 0.14726163, -0.23246839]]), array([[0.58959629, 0.70061509]])]
derivative [array([[-0.02431958, -0.        , -0.0243195

derivative [array([[-0.02431958, -0.        , -0.02431958]]), array([[ 0.04194088, -0.16514538]]), array([[-0.32391856, -0.28009909]])]
derivative [array([[-0.01006105, -0.01006105, -0.01006105]]), array([[-0.04104802, -0.23988655]]), array([[1.1176476 , 0.74057675]])]
derivative [array([[0.        , 0.01274629, 0.01274629]]), array([[ 0.06471013, -0.3105215 ]]), array([[-0.33180615, -0.27783489]])]
derivative [array([[-0.        , -0.        , -0.01504306]]), array([[ 0.14726163, -0.23246839]]), array([[0.58959629, 0.70061509]])]
derivative [array([[-0.02431958, -0.        , -0.02431958]]), array([[ 0.04194088, -0.16514538]]), array([[-0.32391856, -0.28009909]])]
derivative [array([[-0.01006105, -0.01006105, -0.01006105]]), array([[-0.04104802, -0.23988655]]), array([[1.1176476 , 0.74057675]])]
derivative [array([[0.        , 0.01274629, 0.01274629]]), array([[ 0.06471013, -0.3105215 ]]), array([[-0.33180615, -0.27783489]])]
derivative [array([[-0.        , -0.        , -0.01504306]])

In [20]:
# TODO
# Test the trained model
test_data =np.array([[0,0],[0,1],[1,0],[1,1]])
predictions = multi_neuron.inference(test_data)
print(predictions)
# END TODO

[[0.70061509]
 [0.72216511]
 [0.71990091]
 [0.74057675]]


In [21]:
# TODO
# Call accuracy function and provide necessary inputs to calculate accuracy
accuracy_value = accuracy(predictions,targets)
print(accuracy_value)
# END TODO

[[ True]
 [False]
 [ True]
 [False]]
0.5


In [376]:
class MLP_implementation():
    def __init__(self,
        hidden_layers,
        hidden_layers_size,
        hidden_activation_func,
        output_activation_function,
        loss_function,
        output_dim,
    ):
        self.hidden_layers = hidden_layers
        self.hidden_layers_size = hidden_layers_size
        self.hidden_activation_func = hidden_activation_func
        self.loss_function = loss_function
        self.output_activation_function = output_activation_function
        self.output_dim=output_dim
        # TODO
        # Implement your MLP model 
        self.neuron_weights = None
        # END TODO

    def forward_pass(self, x):
        # TODO
        # Implement forward propagation
        output = None
        # self.outputs records output of each layer includes initial input
        self.outputs={}
        # self.outputs_before_activate records the input of each activation function(Relu) to compute derivation
        self.outputs_before_activate={}
        
        #initiazation:
        if self.neuron_weights==None:
            self.neuron_weights={}
            # first hidden layer
            self.neuron_weights["W0"]=xavier_initialization(x.shape[1]-1,self.hidden_layers_size)
            self.neuron_weights["B0"]=xavier_initialization(self.hidden_layers_size-1,1)
            #following hidden layer
            for j in range(self.hidden_layers-1):
                self.neuron_weights["W"+str(j+1)]=xavier_initialization(self.hidden_layers_size-1,self.hidden_layers_size)
                self.neuron_weights["B"+str(j+1)]=xavier_initialization(self.hidden_layers_size-1,1)
            #output layer
            self.neuron_weights["W"+str(self.hidden_layers)]=xavier_initialization(self.hidden_layers_size-1,self.output_dim)
            self.neuron_weights["B"+str(self.hidden_layers)]=xavier_initialization(self.output_dim-1,1) 
            print("initial weights",self.neuron_weights)
        
        input_next=x
        self.outputs["X"]=x
        for i in range(self.hidden_layers):
            #full connection
            H=np.dot(input_next,self.neuron_weights["W"+str(i)].T)+self.neuron_weights["B"+str(i)]
            #activation layer
            input_next=self.hidden_activation_func.forward(H)
            self.outputs_before_activate["Output_b"+str(i)]=H
            self.outputs["Output"+str(i)]=input_next
    
        #output layer
        input_next=np.dot(input_next,self.neuron_weights["W"+str(self.hidden_layers)].T)+self.neuron_weights["B"+str(self.hidden_layers)]
        self.outputs_before_activate["Output_b"+str(self.hidden_layers)]=input_next
        output_ac=self.output_activation_function
        output=output_ac.forward(input_next)
        self.outputs["Output"+str(self.hidden_layers)]=output
        # END TODO
        return output

    def backward_pass(self, x):
        # TODO
        # Perform backpropagation by calculating derivative
        output = {}
        input_next=x
        #hidden layers
        for i in range(self.hidden_layers):
            
            #gradient of activation function
            #input_next: previous gradient
            if i==0:
                d_act=self.output_activation_function.backward(input_next)
            else:
                g={}
                g["gradient"]=input_next
                g["input"]=self.outputs_before_activate["Output_b"+str(self.hidden_layers-i)]
                d_act=self.hidden_activation_func.backward(g)
    
            dW=np.dot(self.outputs["Output"+str(self.hidden_layers-i-1)].T,d_act)/(input_next.shape[0])
            db=np.sum(input_next, axis=0, keepdims=True)/(input_next.shape[0])
            output["d_W" + str(self.hidden_layers-i)] = dW
            output["d_B" + str(self.hidden_layers-i)] = db
            input_next=np.dot(d_act,self.neuron_weights["W"+str(self.hidden_layers-i)])
        #input layers
        g={}
        g["gradient"]=input_next
#         print("g[gradient]",g["gradient"])
        g["input"]=self.outputs_before_activate["Output_b0"]
#         print("g[input]",g["input"])
        d_act=self.hidden_activation_func.backward(g)
#         print(d_act)
        dW_0=np.dot(self.outputs["X"].T,d_act)/(input_next.shape[0])
        db_0=np.sum(input_next, axis=0, keepdims=True) /(input_next.shape[0])
        output["d_W" + str(0)] = dW_0
        output["d_B" + str(0)] = db_0
        # END TODO
        return output

    def update_parameter(self,derivative, learning_rate = 0.1):
        # TODO
        # Perform weight update
        for i in range(self.hidden_layers+1):
            self.neuron_weights["W"+str(i)]-=learning_rate*(derivative["d_W"+str(i)].T)
            self.neuron_weights["B"+str(i)]-=learning_rate*derivative["d_B"+str(i)]
        # END TODO

    def train(self, input_data, targets):
        # TODO
        # Call the necessary functions to train the model for the given task
        # Complete the rest of the code to correctly train the model
        # input_data.shape[0] number of batch
        for j in range(1000):
            for i in range(input_data.shape[0]):
                Y=self.forward_pass(input_data[i])
#                 print(Y)
                # loss
                loss=self.loss_function
                loss_value=loss.forward(Y,targets[i])
                loss_gradient=loss.backward()
#                 print(loss_gradient)
#                 print("lookloss",i,loss_value)

                #backpropagation
                derivative=self.backward_pass(loss_gradient)
                self.update_parameter(derivative)
#             print(self.neuron_weights)
        # END TODO

    def inference(self, input_data):
        # TODO
        # Test the trained model
        output=self.forward_pass(input_data)
        # END TODO
        return output

In [23]:
# You need to use the same implementation that you do in the previous task, if you need.
# You only need to include ReLU activation function in your implementation.

class relu_activation_function():

    def forward(self, input_data):
        output = None
        
        # TODO
        # implement ReLU function for the input_data
        self.input=input_data
        output= np.maximum(0,input_data)
        # END TODO
        return output
    
    def backward(self, gradients):
        # calculate the gradients with help of the derivative
#         print(self.input)
        gradients["gradient"][gradients["input"]<=0]=0
#         gradients[self.input<=0]=0
        return gradients["gradient"]

In [38]:
relu = relu_activation_function()
xor_mlp = MLP_implementation(3,3,hidden_activation_func=relu,output_activation_function=sigmoid_activation_function(),loss_function=binary_cross_entropy(),output_dim=1,batchsize=1,epoch=1000,lr=0.1)

In [39]:
# Train the same model with ReLU activation function
# TODO
input_data =np.array([[0,0],[0,1],[1,0],[1,1]])
targets = np.array([[0],[1],[1],[0]])
# train the corresponding single neuron 
xor_mlp.train(input_data, targets)
# END TODO

initial weights {'W0': array([[-0.71180612, -1.06735388],
       [-0.31816569,  1.18807311],
       [-0.98108998,  0.26570501]]), 'B0': array([[ 0.08597418, -0.04855642,  0.91983076]]), 'W1': array([[ 0.55870543, -0.24105142, -0.57003942],
       [-0.75450952,  0.49147963,  0.90745664],
       [-0.05909676,  0.39108245,  0.66103739]]), 'B1': array([[-0.11827187,  0.49167872, -0.99604562]]), 'W2': array([[-0.31308858,  0.25518746, -0.12595317],
       [ 0.16640327, -0.90956721, -0.60625526],
       [-0.61807156,  0.71333632, -0.08710673]]), 'B2': array([[0.12923754, 0.81348325, 0.67842894]]), 'W3': array([[0.62252996, 0.84237991, 1.0801661 ]]), 'B3': array([[1.5755348]])}
loss 0 0 [3.58924579]
loss 0 1 [0.34327902]
loss 0 2 [1.05470851]
loss 0 3 [1.00375622]
loss 1 0 [0.35222798]
loss 1 1 [0.57900745]
loss 1 2 [1.18862314]
loss 1 3 [0.71162592]
loss 2 0 [0.48750097]
loss 2 1 [1.10828457]
loss 2 2 [0.57523303]
loss 2 3 [0.7881176]
loss 3 0 [0.54580928]
loss 3 1 [1.05397957]
loss 3 2 [0.6

loss 95 1 [0.03018234]
loss 95 2 [0.01303608]
loss 95 3 [0.0545872]
loss 96 0 [0.04789939]
loss 96 1 [0.02914861]
loss 96 2 [0.04762175]
loss 96 3 [0.01452788]
loss 97 0 [0.01065655]
loss 97 1 [0.02297844]
loss 97 2 [0.05904905]
loss 97 3 [0.05628254]
loss 98 0 [0.04615446]
loss 98 1 [0.02898141]
loss 98 2 [0.00852136]
loss 98 3 [0.07434266]
loss 99 0 [0.05565321]
loss 99 1 [0.02811921]
loss 99 2 [0.05685388]
loss 99 3 [0.00909686]
loss 100 0 [0.00711325]
loss 100 1 [0.02341004]
loss 100 2 [0.07557766]
loss 100 3 [0.05875297]
loss 101 0 [0.04717432]
loss 101 1 [0.02806163]
loss 101 2 [0.01341415]
loss 101 3 [0.04951068]
loss 102 0 [0.04408566]
loss 102 1 [0.02713595]
loss 102 2 [0.0447718]
loss 102 3 [0.01382766]
loss 103 0 [0.01014934]
loss 103 1 [0.02143075]
loss 103 2 [0.05397954]
loss 103 3 [0.04918765]
loss 104 0 [0.04355639]
loss 104 1 [0.0256639]
loss 104 2 [0.01096304]
loss 104 3 [0.04799375]
loss 105 0 [0.04331554]
loss 105 1 [0.02489144]
loss 105 2 [0.0432348]
loss 105 3 [0.0

loss 195 2 [0.02568934]
loss 195 3 [0.0019346]
loss 196 0 [0.00167353]
loss 196 1 [0.00712984]
loss 196 2 [0.03646063]
loss 196 3 [0.02820707]
loss 197 0 [0.02548472]
loss 197 1 [0.00866602]
loss 197 2 [0.00215249]
loss 197 3 [0.02895126]
loss 198 0 [0.02537342]
loss 198 1 [0.00857297]
loss 198 2 [0.02533065]
loss 198 3 [0.0026246]
loss 199 0 [0.00223878]
loss 199 1 [0.00692772]
loss 199 2 [0.03081121]
loss 199 3 [0.02783364]
loss 200 0 [0.02514784]
loss 200 1 [0.00841769]
loss 200 2 [0.0021428]
loss 200 3 [0.0277156]
loss 201 0 [0.02504134]
loss 201 1 [0.00832921]
loss 201 2 [0.02499935]
loss 201 3 [0.00227088]
loss 202 0 [0.00195072]
loss 202 1 [0.00673337]
loss 202 2 [0.03040782]
loss 202 3 [0.02746984]
loss 203 0 [0.02481962]
loss 203 1 [0.00818331]
loss 203 2 [0.00187776]
loss 203 3 [0.02735388]
loss 204 0 [0.024715]
loss 204 1 [0.00809993]
loss 204 2 [0.0246738]
loss 204 3 [0.00200052]
loss 205 0 [0.00172788]
loss 205 1 [0.00655036]
loss 205 2 [0.030219]
loss 205 3 [0.02711215]
l

loss 300 2 [0.01752892]
loss 300 3 [0.00079421]
loss 301 0 [0.00070321]
loss 301 1 [0.00375301]
loss 301 2 [0.0241823]
loss 301 3 [0.01890532]
loss 302 0 [0.01708918]
loss 302 1 [0.00457435]
loss 302 2 [0.0007995]
loss 302 3 [0.0196572]
loss 303 0 [0.01703555]
loss 303 1 [0.00454777]
loss 303 2 [0.017389]
loss 303 3 [0.00079138]
loss 304 0 [0.00070065]
loss 304 1 [0.00377556]
loss 304 2 [0.02297883]
loss 304 3 [0.01983819]
loss 305 0 [0.01692079]
loss 305 1 [0.00460138]
loss 305 2 [0.00095287]
loss 305 3 [0.01866256]
loss 306 0 [0.01686996]
loss 306 1 [0.00457374]
loss 306 2 [0.01684918]
loss 306 3 [0.00103148]
loss 307 0 [0.00090788]
loss 307 1 [0.00371652]
loss 307 2 [0.02051854]
loss 307 3 [0.01860578]
loss 308 0 [0.01676445]
loss 308 1 [0.00452815]
loss 308 2 [0.00098532]
loss 308 3 [0.0184905]
loss 309 0 [0.01671458]
loss 309 1 [0.00450136]
loss 309 2 [0.01669413]
loss 309 3 [0.00106568]
loss 310 0 [0.00093717]
loss 310 1 [0.00365807]
loss 310 2 [0.02033039]
loss 310 3 [0.01837596

loss 398 0 [0.01306294]
loss 398 1 [0.00324003]
loss 398 2 [0.00055322]
loss 398 3 [0.0149596]
loss 399 0 [0.01303128]
loss 399 1 [0.00322602]
loss 399 2 [0.01301849]
loss 399 3 [0.00066278]
loss 400 0 [0.00058819]
loss 400 1 [0.00262652]
loss 400 2 [0.01586353]
loss 400 3 [0.01434192]
loss 401 0 [0.0129673]
loss 401 1 [0.00320256]
loss 401 2 [0.00058029]
loss 401 3 [0.01430819]
loss 402 0 [0.01293683]
loss 402 1 [0.00318869]
loss 402 2 [0.01292421]
loss 402 3 [0.0006328]
loss 403 0 [0.00056203]
loss 403 1 [0.00259628]
loss 403 2 [0.01574882]
loss 403 3 [0.01423829]
loss 404 0 [0.01287368]
loss 404 1 [0.00316579]
loss 404 2 [0.00055485]
loss 404 3 [0.01420501]
loss 405 0 [0.01284361]
loss 405 1 [0.00315224]
loss 405 2 [0.01283115]
loss 405 3 [0.00060545]
loss 406 0 [0.00053811]
loss 406 1 [0.00256676]
loss 406 2 [0.01564235]
loss 406 3 [0.014136]
loss 407 0 [0.01278126]
loss 407 1 [0.00312988]
loss 407 2 [0.00058248]
loss 407 3 [0.01410322]
loss 408 0 [0.01275164]
loss 408 1 [0.0031166

loss 502 2 [0.01268083]
loss 502 3 [0.01146643]
loss 503 0 [0.010369]
loss 503 1 [0.00243374]
loss 503 2 [0.00043146]
loss 503 3 [0.01144469]
loss 504 0 [0.01034935]
loss 504 1 [0.00242539]
loss 504 2 [0.01034114]
loss 504 3 [0.00047181]
loss 505 0 [0.00042064]
loss 505 1 [0.00197703]
loss 505 2 [0.01260688]
loss 505 3 [0.01139961]
loss 506 0 [0.01030861]
loss 506 1 [0.00241159]
loss 506 2 [0.00041635]
loss 506 3 [0.0113781]
loss 507 0 [0.01028917]
loss 507 1 [0.00240339]
loss 507 2 [0.01028105]
loss 507 3 [0.00045548]
loss 508 0 [0.00040626]
loss 508 1 [0.00195917]
loss 508 2 [0.01253372]
loss 508 3 [0.01135811]
loss 509 0 [0.01024883]
loss 509 1 [0.00238985]
loss 509 2 [0.00043247]
loss 509 3 [0.01131222]
loss 510 0 [0.01022962]
loss 510 1 [0.0023818]
loss 510 2 [0.01022159]
loss 510 3 [0.00047288]
loss 511 0 [0.00042157]
loss 511 1 [0.00194163]
loss 511 2 [0.01246141]
loss 511 3 [0.01126815]
loss 512 0 [0.0101898]
loss 512 1 [0.00236849]
loss 512 2 [0.00041724]
loss 512 3 [0.0112471

loss 615 3 [0.00036242]
loss 616 0 [0.00032402]
loss 616 1 [0.00153331]
loss 616 2 [0.01035064]
loss 616 3 [0.00936055]
loss 617 0 [0.00846561]
loss 617 1 [0.00187091]
loss 617 2 [0.00032135]
loss 617 3 [0.00934592]
loss 618 0 [0.00845239]
loss 618 1 [0.00186585]
loss 618 2 [0.00844682]
loss 618 3 [0.00035223]
loss 619 0 [0.00031501]
loss 619 1 [0.00152228]
loss 619 2 [0.01030091]
loss 619 3 [0.0093156]
loss 620 0 [0.00842497]
loss 620 1 [0.00185748]
loss 620 2 [0.0003125]
loss 620 3 [0.0093011]
loss 621 0 [0.00841186]
loss 621 1 [0.00185249]
loss 621 2 [0.00840634]
loss 621 3 [0.00034261]
loss 622 0 [0.00030649]
loss 622 1 [0.00151141]
loss 622 2 [0.01025161]
loss 622 3 [0.00927104]
loss 623 0 [0.00838469]
loss 623 1 [0.00184424]
loss 623 2 [0.00030412]
loss 623 3 [0.00925667]
loss 624 0 [0.0083717]
loss 624 1 [0.00183932]
loss 624 2 [0.00836623]
loss 624 3 [0.00033351]
loss 625 0 [0.00029843]
loss 625 1 [0.00150071]
loss 625 2 [0.01020275]
loss 625 3 [0.00922687]
loss 626 0 [0.008344

loss 724 1 [0.00126483]
loss 724 2 [0.00880874]
loss 724 3 [0.00796679]
loss 725 0 [0.00720563]
loss 725 1 [0.00154356]
loss 725 2 [0.0002647]
loss 725 3 [0.00795615]
loss 726 0 [0.00719601]
loss 726 1 [0.00154001]
loss 726 2 [0.00719194]
loss 726 3 [0.00029047]
loss 727 0 [0.0002602]
loss 727 1 [0.0012571]
loss 727 2 [0.00877256]
loss 727 3 [0.00793581]
loss 728 0 [0.00717605]
loss 728 1 [0.00153415]
loss 728 2 [0.00026701]
loss 728 3 [0.00792352]
loss 729 0 [0.00716651]
loss 729 1 [0.00153064]
loss 729 2 [0.00716247]
loss 729 3 [0.00029298]
loss 730 0 [0.00026242]
loss 730 1 [0.00124947]
loss 730 2 [0.00873667]
loss 730 3 [0.00790163]
loss 731 0 [0.00714672]
loss 731 1 [0.00152485]
loss 731 2 [0.00026059]
loss 731 3 [0.00789116]
loss 732 0 [0.00713725]
loss 732 1 [0.00152138]
loss 732 2 [0.00713324]
loss 732 3 [0.00028599]
loss 733 0 [0.00025622]
loss 733 1 [0.00124194]
loss 733 2 [0.00870105]
loss 733 3 [0.00786943]
loss 734 0 [0.00711761]
loss 734 1 [0.00151566]
loss 734 2 [0.00025

loss 836 3 [0.00689819]
loss 837 0 [0.00623947]
loss 837 1 [0.00129149]
loss 837 2 [0.00623639]
loss 837 3 [0.00024187]
loss 838 0 [0.00021696]
loss 838 1 [0.00105467]
loss 838 2 [0.0076083]
loss 838 3 [0.00688152]
loss 839 0 [0.00622441]
loss 839 1 [0.00128728]
loss 839 2 [0.00021567]
loss 839 3 [0.00687354]
loss 840 0 [0.00621719]
loss 840 1 [0.00128476]
loss 840 2 [0.00621412]
loss 840 3 [0.00023694]
loss 841 0 [0.00021257]
loss 841 1 [0.0010492]
loss 841 2 [0.00758117]
loss 841 3 [0.00685699]
loss 842 0 [0.00620223]
loss 842 1 [0.0012806]
loss 842 2 [0.00021133]
loss 842 3 [0.00684906]
loss 843 0 [0.00619506]
loss 843 1 [0.00127811]
loss 843 2 [0.00619201]
loss 843 3 [0.0002322]
loss 844 0 [0.00020835]
loss 844 1 [0.00104378]
loss 844 2 [0.00755421]
loss 844 3 [0.00683262]
loss 845 0 [0.00618019]
loss 845 1 [0.00127399]
loss 845 2 [0.00020717]
loss 845 3 [0.00682474]
loss 846 0 [0.00617307]
loss 846 1 [0.00127152]
loss 846 2 [0.00617004]
loss 846 3 [0.00022765]
loss 847 0 [0.000204

loss 944 1 [0.00111883]
loss 944 2 [0.00018295]
loss 944 3 [0.00610443]
loss 945 0 [0.00552175]
loss 945 1 [0.00111689]
loss 945 2 [0.00551931]
loss 945 3 [0.00020115]
loss 946 0 [0.00018065]
loss 946 1 [0.00091236]
loss 946 2 [0.00673435]
loss 946 3 [0.00609134]
loss 947 0 [0.00550991]
loss 947 1 [0.00111368]
loss 947 2 [0.00017974]
loss 947 3 [0.00608506]
loss 948 0 [0.00550423]
loss 948 1 [0.00111176]
loss 948 2 [0.00550181]
loss 948 3 [0.00019763]
loss 949 0 [0.00017751]
loss 949 1 [0.00090818]
loss 949 2 [0.00671301]
loss 949 3 [0.00607204]
loss 950 0 [0.00549246]
loss 950 1 [0.00110858]
loss 950 2 [0.00017663]
loss 950 3 [0.0060658]
loss 951 0 [0.00548682]
loss 951 1 [0.00110668]
loss 951 2 [0.00548755]
loss 951 3 [0.00020404]
loss 952 0 [0.00018323]
loss 952 1 [0.00090404]
loss 952 2 [0.00669181]
loss 952 3 [0.00605287]
loss 953 0 [0.00547513]
loss 953 1 [0.00110353]
loss 953 2 [0.00018229]
loss 953 3 [0.00604668]
loss 954 0 [0.00546952]
loss 954 1 [0.00110164]
loss 954 2 [0.005

In [40]:
# TODO
# Call accuracy function and provide necessary inputs to calculate accuracy
test_data =np.array([[0,0],[0,1],[1,0],[1,1]])
test_target=np.array([[0],[1],[1],[0]])
predictions = xor_mlp.inference(test_data)
print(predictions)
accuracy_value = accuracy(predictions,test_target)
print(accuracy_value)
# END TODO

[[0.00520314]
 [0.99982064]
 [0.99905327]
 [0.00520314]]
[[ True]
 [ True]
 [ True]
 [ True]]
1.0


In [41]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
#one hot encoding and batch splitting(one sample a batch)
y_1=np.identity(3)[y]
X_1=X

In [44]:
#process the data:normalization
X_2=(X-np.min(X,axis=0,keepdims=True))/(np.max(X,axis=0,keepdims=True)-np.min(X,axis=0,keepdims=True))

In [45]:
# Split the dataset into training and test dataset
X_train, X_test, y_train, y_test = train_test_split(X_2, y_1, test_size=0.2)

In [55]:
X_train2=X_train.reshape(120,1,4)
y_train2=y_train.reshape(120,1,3)

In [281]:
y_train3=y_train.reshape(12,10,3)
X_train3=X_train.reshape(12,10,4)

In [40]:
X_train1=np.expand_dims(X_train,0).repeat(10,axis=0)

In [46]:
y_train1=np.expand_dims(y_train,0).repeat(10,axis=0)

In [28]:
class MLP_implementation():
    def __init__(self,
        hidden_layers,
        hidden_layers_size,
        hidden_activation_func,
        output_activation_function,
        loss_function,
        output_dim,
        batchsize,
        epoch,
        lr,
    ):
        self.hidden_layers = hidden_layers
        self.hidden_layers_size = hidden_layers_size
        self.hidden_activation_func = hidden_activation_func
        self.loss_function = loss_function
        self.output_activation_function = output_activation_function
        self.output_dim=output_dim
        self.batchsize=batchsize
        self.epoch=epoch
        #learning rate
        self.lr=lr
        # TODO
        # Implement your MLP model 
        self.neuron_weights = None
        # END TODO

    def forward_pass(self, x):
        # TODO
        # Implement forward propagation
        output = None
        # self.outputs records output of each layer includes initial input
        self.outputs={}
        # self.outputs_before_activate
        self.outputs_before_activate={}
        #initiazation:
        if self.neuron_weights==None:
            self.neuron_weights={}
            # first hidden layer
            self.neuron_weights["W0"]=xavier_initialization(x.shape[1]-1,self.hidden_layers_size)
            self.neuron_weights["B0"]=xavier_initialization(self.hidden_layers_size-1,1)
            #following hidden layer
            for j in range(self.hidden_layers-1):
                self.neuron_weights["W"+str(j+1)]=xavier_initialization(self.hidden_layers_size-1,self.hidden_layers_size)
                self.neuron_weights["B"+str(j+1)]=xavier_initialization(self.hidden_layers_size-1,1)
            #output layer
            self.neuron_weights["W"+str(self.hidden_layers)]=xavier_initialization(self.hidden_layers_size-1,self.output_dim)
            self.neuron_weights["B"+str(self.hidden_layers)]=xavier_initialization(self.output_dim-1,1) 
            print("initial weights",self.neuron_weights)
        
        input_next=x
        self.outputs["X"]=x
        for i in range(self.hidden_layers):
            #full connection
            H=np.dot(input_next,self.neuron_weights["W"+str(i)].T)+self.neuron_weights["B"+str(i)]
            #hidden layer activation
            input_next=self.hidden_activation_func.forward(H)
            self.outputs_before_activate["Output_b"+str(i)]=H
            self.outputs["Output"+str(i)]=input_next
        
        #output layer
        input_next=np.dot(input_next,self.neuron_weights["W"+str(self.hidden_layers)].T)+self.neuron_weights["B"+str(self.hidden_layers)]
        self.outputs_before_activate["Output_b"+str(self.hidden_layers)]=input_next
        #output activation
        output_ac=self.output_activation_function
        output=output_ac.forward(input_next)
        self.outputs["Output"+str(self.hidden_layers)]=output
        # END TODO
        return output

    def backward_pass(self, x):
        # TODO
        # Perform backpropagation by calculating derivative
        #output:record the derivation of weights and biases
        output = {}
        input_next=x
        #hidden layers
        for i in range(self.hidden_layers):
            
            #gradient of activation function
            #input_next: previous gradient
            if i==0:
                #output layer
                d_act=self.output_activation_function.backward(input_next)
            else:
                g={}
                g["gradient"]=input_next
                #compute the relu derivation
                g["input"]=self.outputs_before_activate["Output_b"+str(self.hidden_layers-i)]
                d_act=self.hidden_activation_func.backward(g)
    
            #input_next.shape[0]=batch size
            dW=np.dot(self.outputs["Output"+str(self.hidden_layers-i-1)].T,d_act)/(input_next.shape[0])
            db=np.sum(input_next, axis=0, keepdims=True)/(input_next.shape[0])
            output["d_W" + str(self.hidden_layers-i)] = dW
            output["d_B" + str(self.hidden_layers-i)] = db
            input_next=np.dot(d_act,self.neuron_weights["W"+str(self.hidden_layers-i)])
            
        #input layers
        g={}
        g["gradient"]=input_next
        g["input"]=self.outputs_before_activate["Output_b0"]
        d_act=self.hidden_activation_func.backward(g)
        dW_0=np.dot(self.outputs["X"].T,d_act)/(input_next.shape[0])
        db_0=np.sum(input_next, axis=0, keepdims=True) /(input_next.shape[0])
        output["d_W" + str(0)] = dW_0
        output["d_B" + str(0)] = db_0
        
        # END TODO
        return output

    def update_parameter(self,derivative, learning_rate):
        # TODO
        # Perform weight update
        for i in range(self.hidden_layers+1):
            self.neuron_weights["W"+str(i)]-=learning_rate*(derivative["d_W"+str(i)].T)
            self.neuron_weights["B"+str(i)]-=learning_rate*derivative["d_B"+str(i)]
        # END TODO

    def train(self, input_data, targets):
        # TODO
        # Call the necessary functions to train the model for the given task
        # Complete the rest of the code to correctly train the model
        # input_data.shape[0] number of batch
        for j in range(self.epoch):
            randnum=np.random.randint(0,10)
            np.random.seed(randnum)
            np.random.shuffle(input_data)
            np.random.seed(randnum)
            np.random.shuffle(targets)
            max_batch = int(input_data.shape[0] / self.batchsize)
            for i in range(max_batch):
                batch_data=input_data[i*self.batchsize:(i+1)*self.batchsize]
                batch_label=targets[i*self.batchsize:(i+1)*self.batchsize]
                Y=self.forward_pass(batch_data)
                
                # loss
                loss=self.loss_function
                loss_value=loss.forward(Y,batch_label)
                print("loss",j,i,loss_value)
                loss_gradient=loss.backward()

                #backpropagation
                derivative=self.backward_pass(loss_gradient)
                self.update_parameter(derivative,learning_rate=self.lr)
        # END TODO

    def inference(self, input_data):
        # TODO
        # Test the trained model
        output=self.forward_pass(input_data)
        # END TODO
        return output

In [62]:
iris_mlp = MLP_implementation(4,24,hidden_activation_func=relu,output_activation_function=softmax_activation_function(),loss_function=cross_entropy_loss(),output_dim=3,batchsize=10,epoch=300,lr=1e-4)
iris_mlp.train(X_train, y_train)

initial weights {'W0': array([[-0.03006989,  0.29666581, -0.2436217 ,  0.11763661],
       [ 0.02609952,  0.09820273, -0.22125181, -0.06366503],
       [ 0.00660118, -0.0948877 , -0.2003638 ,  0.39721802],
       [-0.08311279, -0.27324811,  0.40105209, -0.07822857],
       [-0.33968194, -0.37737838,  0.39569621, -0.1906049 ],
       [ 0.42437507, -0.15333126, -0.10719642,  0.38835665],
       [ 0.16778959,  0.26171225,  0.18412291,  0.40834357],
       [-0.10732173, -0.31794029, -0.22695795, -0.32906187],
       [ 0.20678174,  0.04904013, -0.05593227,  0.29085326],
       [ 0.05736291,  0.1396297 , -0.09770101,  0.12997931],
       [-0.35417989, -0.06404829, -0.00450538, -0.07751518],
       [-0.13914366,  0.16941869, -0.16395647, -0.28610237],
       [-0.03654191, -0.33065165,  0.20750804,  0.07960777],
       [-0.06639457, -0.42694798,  0.04554209, -0.35211114],
       [ 0.0741938 , -0.31074479, -0.42195355, -0.0268787 ],
       [-0.25813525,  0.29441863, -0.34137784, -0.38959956],
 

loss 19 6 1.0199884134839063
loss 19 7 1.449166086365398
loss 19 8 1.235602490025253
loss 19 9 1.0192306910352276
loss 19 10 1.1762811303977878
loss 19 11 1.2349314000869018
loss 20 0 1.3134145832724164
loss 20 1 1.3126528349708932
loss 20 2 1.0234472913581862
loss 20 3 1.2336290267058374
loss 20 4 1.092575416695026
loss 20 5 1.3802053489848032
loss 20 6 1.308670612938362
loss 20 7 1.1636958694070887
loss 20 8 1.386731724324021
loss 20 9 1.1634167589825706
loss 20 10 1.2275844317929079
loss 20 11 1.2412239445396662
loss 21 0 1.2411950119503201
loss 21 1 1.4535398870010565
loss 21 2 1.232411235580602
loss 21 3 1.2336323133774596
loss 21 4 1.1635526560058547
loss 21 5 1.0115068800743319
loss 21 6 0.8780777773179869
loss 21 7 1.2397693180345586
loss 21 8 1.3059850114826952
loss 21 9 1.157316601637497
loss 21 10 1.5291620549298726
loss 21 11 1.3779541530495105
loss 22 0 1.2348001919107954
loss 22 1 1.2327212867079222
loss 22 2 1.2325967165465415
loss 22 3 1.4482491618593218
loss 22 4 1.168

loss 56 2 1.0519159285727993
loss 56 3 1.5441000732313037
loss 56 4 1.3565656561227333
loss 56 5 0.9328320141646568
loss 56 6 1.2340783583046444
loss 56 7 1.2405185913183874
loss 56 8 0.9958072442327557
loss 56 9 1.4188714864206702
loss 56 10 1.115046996565734
loss 56 11 1.1251875699150404
loss 57 0 1.1227735046131602
loss 57 1 1.2341947095048638
loss 57 2 1.4775950685251464
loss 57 3 1.2402302267517087
loss 57 4 1.299043821083019
loss 57 5 1.228073277231843
loss 57 6 1.0553480953435406
loss 57 7 0.994315435351998
loss 57 8 1.1190590278012582
loss 57 9 1.428289149886847
loss 57 10 0.9830351413314761
loss 57 11 0.9238642882422228
loss 58 0 0.8714877143760955
loss 58 1 1.118597345462934
loss 58 2 1.2907673987406099
loss 58 3 1.2939815212598722
loss 58 4 1.0601504801985586
loss 58 5 1.1743840148658522
loss 58 6 0.993698308378072
loss 58 7 1.5933393440121495
loss 58 8 1.1713978608074072
loss 58 9 1.175307731807273
loss 58 10 1.2252376197628392
loss 58 11 1.1201187862842525
loss 59 0 1.1069

loss 81 6 1.2591013540729366
loss 81 7 1.041083488668851
loss 81 8 1.2581665402305533
loss 81 9 0.9887240622460011
loss 81 10 1.1986436235244136
loss 81 11 1.1385695565693847
loss 82 0 1.042723344979702
loss 82 1 1.1989727706743751
loss 82 2 1.2105526782217706
loss 82 3 1.1566779951357389
loss 82 4 1.1501308619532904
loss 82 5 0.8777742181335035
loss 82 6 1.08783972004824
loss 82 7 1.2592144527315565
loss 82 8 1.2504706682183684
loss 82 9 1.1410430224231374
loss 82 10 1.1399516087211656
loss 82 11 1.2383080117506218
loss 83 0 1.134102627898305
loss 83 1 1.086190994036276
loss 83 2 1.2523637263857086
loss 83 3 1.1302471869919883
loss 83 4 0.9864659149036233
loss 83 5 1.2084011908163383
loss 83 6 1.3737029261619724
loss 83 7 1.0902552304863717
loss 83 8 1.0302389740164828
loss 83 9 1.3641900441937018
loss 83 10 1.099449464254036
loss 83 11 0.9861901715061748
loss 84 0 1.2594853887651711
loss 84 1 1.0940224001349113
loss 84 2 1.2061964199213309
loss 84 3 1.2509048819907265
loss 84 4 1.140

loss 108 4 1.1210666185456515
loss 108 5 0.9723515763029061
loss 108 6 1.167182584247107
loss 108 7 1.2057656018571024
loss 108 8 1.0687692670141038
loss 108 9 1.1735422433053624
loss 108 10 0.978387084305343
loss 108 11 1.3697838574099608
loss 109 0 1.0731526043504112
loss 109 1 1.070894808909502
loss 109 2 1.087512455318241
loss 109 3 1.2698206658147129
loss 109 4 1.3933294804031842
loss 109 5 1.0612561426657632
loss 109 6 1.0236401996887896
loss 109 7 1.0647920117804872
loss 109 8 1.3002748891142624
loss 109 9 0.9906453602100225
loss 109 10 0.967714816435449
loss 109 11 1.120859032903873
loss 110 0 0.9216480722821837
loss 110 1 1.1488765136716288
loss 110 2 1.118843238693543
loss 110 3 1.0734933181714805
loss 110 4 1.2430313114110219
loss 110 5 1.207155046806498
loss 110 6 1.0796206635748795
loss 110 7 1.2103195860891232
loss 110 8 1.0076113536444906
loss 110 9 1.0799905862199242
loss 110 10 1.1857238758181754
loss 110 11 1.1352271384864092
loss 111 0 1.1071295994389128
loss 111 1 1

...

loss 203 0 0.9603368685715352
loss 203 1 1.0249395150414429
loss 203 2 1.0906383532538844
loss 203 3 1.0180893422008572
loss 203 4 1.106127225726383
loss 203 5 1.0640467570896275
loss 203 6 1.0484996707215504
loss 203 7 0.9419167723915276
loss 203 8 1.0564210691000728
loss 203 9 1.0395153022720047
loss 203 10 1.084118367433515
loss 203 11 1.0064653354723396
loss 204 0 1.0118726284300659
loss 204 1 1.0477705806077966
loss 204 2 0.990598477065667
loss 204 3 1.1427620679903874
loss 204 4 1.0527174189762014
loss 204 5 1.0231357122626912
loss 204 6 0.9660599792168252
loss 204 7 1.0903832480985225
loss 204 8 1.0053586819849032
loss 204 9 1.0558850736516028
loss 204 10 1.0422092542745496
loss 204 11 1.0023315528453582
loss 205 0 1.0590231319525618
loss 205 1 1.019137541215739
loss 205 2 1.0398688677835282
loss 205 3 1.0759016518587263
loss 205 4 0.9517481011685771
loss 205 5 0.975923359026935
loss 205 6 1.1031703896175664
loss 205 7 1.0167404076279944
loss 205 8 1.0520854852764967
loss 205 9 

loss 226 0 1.0531358438191556
loss 226 1 1.0140170355330114
loss 226 2 0.9883095347973765
loss 226 3 1.0527538439437616
loss 226 4 1.0324396791021762
loss 226 5 1.0374727029494373
loss 226 6 0.9412907479313126
loss 226 7 1.0173759874946302
loss 226 8 1.0155456699152414
loss 226 9 1.0386045426438302
loss 226 10 1.0380113915706484
loss 226 11 1.0203027030738991
loss 227 0 1.00001933707644
loss 227 1 0.982106786973063
loss 227 2 1.0930021524833173
loss 227 3 1.0007615718791647
loss 227 4 1.028316652113126
loss 227 5 1.015829885567977
loss 227 6 1.0031421119808903
loss 227 7 0.99055824602481
loss 227 8 1.002286079957741
loss 227 9 0.9797685048307112
loss 227 10 1.0997024122351058
loss 227 11 1.045971705234479
loss 228 0 1.1003073438382716
loss 228 1 1.0062411295732874
loss 228 2 1.01992010374074
loss 228 3 1.0428174926309066
loss 228 4 0.9601928888020472
loss 228 5 1.0338375221271843
loss 228 6 0.9974226307026297
loss 228 7 1.0315044452040945
loss 228 8 1.010859345091936
loss 228 9 0.99950

loss 249 0 1.0299492740961087
loss 249 1 1.0089183684414942
loss 249 2 1.0269566662314535
loss 249 3 0.9840842067124749
loss 249 4 1.0155484437300004
loss 249 5 0.9983158757896412
loss 249 6 1.0181433106023567
loss 249 7 1.0302954990906177
loss 249 8 0.993745338749954
loss 249 9 0.9876947795878259
loss 249 10 1.0117454813684412
loss 249 11 0.9982991976335391
loss 250 0 1.0220409724162534
loss 250 1 1.008362309166561
loss 250 2 1.0137673672986232
loss 250 3 1.0282297294352483
loss 250 4 0.9976845801940876
loss 250 5 0.9899230625504917
loss 250 6 1.0075482280973636
loss 250 7 1.0245195975400134
loss 250 8 0.990911694964671
loss 250 9 1.0197042813039476
loss 250 10 1.0047071780253836
loss 250 11 0.9910590933351099
loss 251 0 1.010929781751738
loss 251 1 0.9967652828581433
loss 251 2 1.0106530782512304
loss 251 3 0.9850320341840131
loss 251 4 0.9985116266207793
loss 251 5 1.0117805262577275
loss 251 6 1.0040925548163644
loss 251 7 1.0071622870805244
loss 251 8 0.9998731089490016
loss 251 9

loss 275 0 0.9940844188420407
loss 275 1 0.9947124880108881
loss 275 2 1.0126465585157167
loss 275 3 0.9468756561189009
loss 275 4 0.999498295803068
loss 275 5 1.010995875475348
loss 275 6 1.0151939146389801
loss 275 7 1.0197882180286517
loss 275 8 1.0063322488814503
loss 275 9 1.002113533440559
loss 275 10 1.0252725385748154
loss 275 11 0.9745097223151082
loss 276 0 0.9663628362504081
loss 276 1 0.975970256329548
loss 276 2 1.0205207752010885
loss 276 3 0.9627614610689801
loss 276 4 0.987478042086144
loss 276 5 0.989051018622704
loss 276 6 0.9940070841731762
loss 276 7 0.9983990706159693
loss 276 8 1.0196943228903705
loss 276 9 1.0220715715420756
loss 276 10 1.0653289206031848
loss 276 11 0.9995913538954261
loss 277 0 1.0003215241123624
loss 277 1 1.0139734049767997
loss 277 2 0.9762494879737755
loss 277 3 1.0209787025598627
loss 277 4 1.0012206748716992
loss 277 5 0.9937737193150143
loss 277 6 1.0100167904030999
loss 277 7 0.969459881757669
loss 277 8 0.957323723463549
loss 277 9 0.9

loss 298 9 1.0231159023206255
loss 298 10 0.9335617003839186
loss 298 11 0.9736387149591283
loss 299 0 1.0420921249827224
loss 299 1 1.045581110745798
loss 299 2 0.974826907265738
loss 299 3 1.03086802656983
loss 299 4 0.9432895023888976
loss 299 5 0.9378517240407872
loss 299 6 1.0275963781500308
loss 299 7 0.9558561363721536
loss 299 8 0.9804194508962393
loss 299 9 0.9795475010085518
loss 299 10 1.0503679044656042
loss 299 11 1.0109395205550056


In [64]:
predictions_MLP = iris_mlp.inference(X_test.reshape(30,4))
print(predictions_MLP)

[[0.38840077 0.36396298 0.24763625]
 [0.38762181 0.36577764 0.24660055]
 [0.3713309  0.3504807  0.27818841]
 [0.38075941 0.36449104 0.25474955]
 [0.35662906 0.32104284 0.3223281 ]
 [0.34083846 0.19449343 0.46466811]
 [0.32617663 0.3238914  0.34993197]
 [0.38620154 0.3578901  0.25590836]
 [0.31221781 0.2621117  0.42567049]
 [0.31370135 0.27761523 0.40868342]
 [0.32111313 0.30848866 0.37039821]
 [0.39054855 0.37059618 0.23885527]
 [0.32464282 0.31892317 0.35643401]
 [0.32754383 0.32905061 0.34340556]
 [0.3229746  0.22164636 0.45537904]
 [0.38058427 0.3533747  0.26604103]
 [0.31244067 0.29620579 0.39135354]
 [0.33646733 0.19442465 0.46910802]
 [0.30904487 0.29044377 0.40051136]
 [0.39168575 0.36593194 0.24238231]
 [0.3106085  0.27935675 0.41003475]
 [0.32206048 0.23292244 0.44501708]
 [0.32295182 0.22617532 0.45087287]
 [0.31198754 0.28108031 0.40693215]
 [0.36487383 0.33646024 0.29866593]
 [0.33410232 0.1871989  0.47869878]
 [0.31191614 0.29167122 0.39641264]
 [0.32701686 0.32225411 0.35

In [85]:
class softmax_activation_function():

    def forward(self, input_data):
        output = None
        
        # TODO
        # implement Softmax function for the input_data
        x=np.exp(input_data)
        self.input=x
        output=x/np.sum(x,axis=1,keepdims=True)
        # END TODO
        return output
    
    def backward(self, gradients):
        # calculate the gradients with help of the derivative
        output=gradients*(np.sum(self.input,axis=1,keepdims=True)-self.input)/np.sum(self.input,axis=1,keepdims=True)
        return output

In [48]:
class softmax_activation_function():

    def forward(self, input_data):
        output = None
        
        # TODO
        # implement Softmax function for the input_data
        max_e=np.max(input_data)
        self.input=np.exp(input_data-max_e)
        # avoid overflow
        output=np.exp(input_data-max_e)/np.sum(np.exp(input_data-max_e),axis=1,keepdims=True)
        # END TODO
        return output
    
    def backward(self, gradients):
        # calculate the gradients with help of the derivative
        output=gradients*(np.sum(self.input,axis=1,keepdims=True)-self.input)/np.sum(self.input,axis=1,keepdims=True)
        return output

In [49]:
class cross_entropy_loss():

    def forward(self, output, target):
        loss = None
        
        # TODO
        # implement Cross-Entrops loss function for output, target
        self.output=output
        self.target=target
        loss=np.sum(np.sum(-target*np.log(output+1e-6),axis=1),axis=0)/output.shape[0]
        # END TODO
        return loss
    
    def backward(self):
        # calculate the gradients with help of the derivative
        gradient=(-self.target/(self.output+1e-6))
        return gradient

In [65]:
# Call accuracy function and provide necessary inputs to calculate accuracy
t=np.max(predictions_MLP,axis=1,keepdims=True)
b=(t==predictions_MLP)
u=np.array([[0,1,2]])
pre=np.dot(b,u.T)
target=np.dot(y_test,u.T).reshape(30,1)
accuracy_value_MLP = accuracy(pre, target, threshold=0)
print(accuracy_value_MLP)
# END TODO

[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [ True]
 [ True]]
0.6666666666666666


In [66]:
from sklearn.metrics import confusion_matrix 
C=confusion_matrix(target,pre)

In [67]:
C

array([[10,  0,  0],
       [ 0,  1,  9],
       [ 0,  0, 10]], dtype=int64)